In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
from os import path
import cv2
import numpy as np
from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui
import pytesseract
from PIL import Image
from pytesseract import image_to_string
from gtts import gTTS
import os

In [5]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Gabriel Benga\Documents\Python\Python-sistemas\tesseract-tudo\tesseract.exe"
tessdata_config = r'--tessdata-dir "C:\Users\Gabriel Benga\Documents\Python\Python-sistemas\tesseract-tudo\tessdata"'

In [6]:
class RecordVideo(QtCore.QObject):
    image_data = QtCore.pyqtSignal(np.ndarray)

    def __init__(self, camera_port=0, parent=None):
        super().__init__(parent)
        self.camera = cv2.VideoCapture(camera_port)
        
        self.timer = QtCore.QBasicTimer()

    def start_recoding(self):
        self.timer.start(0, self)

    def timerEvent(self, event):
        if (event.timerId() != self.timer.timerId()):
            return
        read, data = self.camera.read()
        if read:
            self.image_data.emit(data)
    
    def saveframe(self):
        read, data = self.camera.read()
        if read:
            cv2.imwrite("a.png", data)
            image = Image.fromarray(data)
            image.load()

            text = pytesseract.image_to_string(image, lang='por', config=tessdata_config)
            print('Texto_Encontrado:', text, len(text))

            if len(text)>0:
                text_to_speech = gTTS(text=text, lang='pt')
                text_to_speech.save('ConverteVoice.mp3')
                os.system('start converteVoice.mp3')

In [7]:
class DetectionWidget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.image = QtGui.QImage()
        self._red = (0, 0, 255)
        self._width = 2
        self._min_size = (30, 30)

    def image_data_slot(self, image_data):
        self.image = self.get_qimage(image_data)
        if self.image.size() != self.size():
            self.setFixedSize(self.image.size())
        self.update()

    def get_qimage(self, image: np.ndarray):
        height, width, color = image.shape
        bytesPerLine = 3 * width
        QImage = QtGui.QImage

        image = QImage(
            image.data,
            width,
            height,
            bytesPerLine,
            QImage.Format_RGB888
        )

        image = image.rgbSwapped()
        return image
    
    def paintEvent(self, event):
        painter = QtGui.QPainter(self)
        painter.drawImage(0, 0, self.image)
        self.image = QtGui.QImage()

In [8]:
class MainWidget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.detection_widget = DetectionWidget()
        self.record_video = RecordVideo()

        image_slot = self.detection_widget.image_data_slot
        self.record_video.image_data.connect(image_slot)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.detection_widget)
        self.run_button = QtWidgets.QPushButton('Inicie aqui')
        layout.addWidget(self.run_button)
        
        self.run_button.clicked.connect(self.record_video.start_recoding)

        self.screenshot = QtWidgets.QPushButton('Aperte aqui para capturar a immagem')
        layout.addWidget(self.screenshot)
        self.screenshot.clicked.connect(self.record_video.saveframe)
        self.setLayout(layout)

In [9]:
def run():
    app = QtWidgets.QApplication(sys.argv)
    main_window = QtWidgets.QMainWindow()
    main_widget = MainWidget()
    main_window.setCentralWidget(main_widget)
    main_window.show()

    sys.exit(app.exec_())

In [10]:
run()

Texto_Encontrado: Espero que os orientadores
gostem do meu projeto
 49


SystemExit: 0

C:\Users\Gabriel Benga\Documents\Python\Python-sistemas\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
